In [1]:
# Configuración servidor base de datos transaccional
# Utilizar el usuario asignado en el curso
db_user = 'Estudiante_19_202413'
db_psswd = 'aabb1122'

In [2]:
source_db_connection_string = 'jdbc:mysql://157.253.236.120:8080/RaSaTransaccional_ETL'

dest_db_connection_string = 'jdbc:mysql://157.253.236.120:8080/Estudiante_19_202413'

# Driver de conexion
path_jar_driver = 'C:\Program Files (x86)\MySQL\Connector J 8.0\mysql-connector-java-8.0.28.jar'

In [242]:
import os 
from pyspark.sql import functions as f, SparkSession, types as t
from pyspark.sql.functions import lit
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.functions import udf, col, length, isnan, when, count, regexp_replace, year, month, dayofmonth, concat, current_date, expr,substring,lower
from pyspark.sql.functions import col, when, length, lit, year, month, dayofmonth, concat, current_date, expr, to_date
import mysql.connector
from pyspark.sql.window import Window
from datetime import datetime
from pyspark.sql import functions as F

In [4]:
#Configuración de la sesión
conf=SparkConf() \
    .set('spark.driver.extraClassPath', path_jar_driver)

spark_context = SparkContext(conf=conf)
sql_context = SQLContext(spark_context)
spark = sql_context.sparkSession
spark.conf.set("spark.sql.hive.convertMetastoreParquet", "false")


C:\Users\estudiante\anaconda3\envs\Tutoriales\lib\site-packages\pyspark\sql\context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


### Conexión y carga de datos

Se define la función para conexión y cargue de dataframes desde la base de datos origen y luego la función para guardar un dataframe en una tabla de la base de datos destino.

In [5]:
def obterner_dataframe_desde_csv(_PATH, _sep):
    return spark.read.load(_PATH, format="csv", sep=_sep, inferSchema="true", header='true')

def obtener_dataframe_de_bd(db_connection_string, sql, db_user, db_psswd):
    df_bd = spark.read.format('jdbc')\
        .option('url', db_connection_string) \
        .option('dbtable', sql) \
        .option('user', db_user) \
        .option('password', db_psswd) \
        .option('driver', 'com.mysql.cj.jdbc.Driver') \
        .load()
    return df_bd

def guardar_db(db_connection_string, df, tabla, db_user, db_psswd):
    df.select('*').write.format('jdbc') \
      .mode('append') \
      .option('url', db_connection_string) \
      .option('dbtable', tabla) \
      .option('user', db_user) \
      .option('password', db_psswd) \
      .option('driver', 'com.mysql.cj.jdbc.Driver') \
      .save()

### Extracción

In [414]:
areas_servicio = obtener_dataframe_de_bd(source_db_connection_string, "FuenteAreasDeServicio_ETL", db_user, db_psswd)
condiciones_pago = obtener_dataframe_de_bd(source_db_connection_string, "FuenteCondicionesDePago_ETL", db_user, db_psswd)
planes_beneficio = obtener_dataframe_de_bd(source_db_connection_string, "FuentePlanesBeneficio_ETL", db_user, db_psswd)
tipos_beneficio = obtener_dataframe_de_bd(source_db_connection_string, "FuenteTiposBeneficio_ETL", db_user, db_psswd)
niveles_servicio = obtener_dataframe_de_bd(source_db_connection_string, "NivelesDeServicio", db_user, db_psswd)

### Transformación

#### Transformación en niveles de servicio

In [415]:
niveles_servicio.count()

3

A niveles de servicio se le quitaron duplicados.

In [416]:
niveles_servicio = niveles_servicio.dropDuplicates()
niveles_servicio.show()

+---------------------+-------------------+---------------+
|IdNivelDeServicio_DWH|IdNivelDeServicio_T|    Descripcion|
+---------------------+-------------------+---------------+
|                    1|                  1|        Nivel 1|
|                    2|                  2|        Nivel 2|
|                    3|                  3|Fuera de la red|
+---------------------+-------------------+---------------+



#### Transformación en areas de servicio

In [417]:
print(areas_servicio.count())
areas_servicio.show(5)
areas_servicio.printSchema()

268444
+------------------+--------------------+-------------+-----------------+----------+------------+-----+--------+-----+
|IdAreaDeServicio_T|NombreAreaDeServicio|IdGeografia_T|          Condado|    Estado|PoblacionAct| Area|Densidad|Fecha|
+------------------+--------------------+-------------+-----------------+----------+------------+-----+--------+-----+
|         100622017|New Jersey - Medi...|        34005|Burlington County|New Jersey|      464269|805.0|   577.0| 2017|
|         100722019|New Jersey  - Med...|        34023| Middlesex County|New Jersey|      860807|311.0|  2768.0| 2019|
|         100922020|New Jersey - Medi...|        34019| Hunterdon County|New Jersey|      129924|430.0|   302.0| 2020|
|         101012018|New Jersey - Medi...|        34031|   Passaic County|New Jersey|      518117|185.0|  2801.0| 2018|
|         101062020|New Jersey - Medi...|        34031|   Passaic County|New Jersey|      518117|185.0|  2801.0| 2020|
+------------------+--------------------+

Se quitaron en la columna "Area" las negativas

In [418]:
areas_negativas = areas_servicio.filter(col("Area") < 0)
areas_negativas.show()

+------------------+--------------------+-------------+------------------+--------------+------------+-------+--------+-----+
|IdAreaDeServicio_T|NombreAreaDeServicio|IdGeografia_T|           Condado|        Estado|PoblacionAct|   Area|Densidad|Fecha|
+------------------+--------------------+-------------+------------------+--------------+------------+-------+--------+-----+
|          24242020|KANSAS25268KS0020...|        20175|     Seward County|        Kansas|       21747| -640.0|    34.0| 2020|
|          21192018|Marketplace Selec...|        42113|   Sullivan County|  Pennsylvania|        5868| -452.0|    13.0| 2018|
|          15162019|Health Alliance P...|        17115|      Macon County|      Illinois|      102432| -580.0|   177.0| 2019|
|          16132019|Health Alliance E...|        17157|   Randolph County|      Illinois|       30142| -575.0|    52.0| 2019|
|          14062020|Health Alliance I...|        17161|Rock Island County|      Illinois|      142909| -427.0|   335.0

In [419]:
areas_servicio = areas_servicio.withColumn("Area", when(col("Area") < 0, col("Area") * -1).otherwise(col("Area")))

In [420]:
incorrectos = areas_servicio.filter(col("PoblacionAct").cast("string").endswith("0001"))

incorrectos.count()

26844

In [421]:
areas_servicio = areas_servicio.withColumn(
    "PoblacionAct",
    when(
        col("PoblacionAct").cast("string").endswith("0001"),
        expr("cast(substr(cast(PoblacionAct as string), 1, length(cast(PoblacionAct as string)) - 4) as int)")
    ).otherwise(col("PoblacionAct"))
)


Se quitaron duplicados u se agregaron las llaves "IdAreaDeServicio_DWH" y "IdGeografia_DWH"

In [422]:
areas_servicio = areas_servicio.dropDuplicates()

areas_servicio = areas_servicio.withColumn('IdAreaDeServicio_DWH', f.monotonically_increasing_id() + 1)
areas_servicio = areas_servicio.withColumn('IdGeografia_DWH', f.monotonically_increasing_id() + 1)

#### Transformación en planes beneficio

In [423]:
print(planes_beneficio.count())
planes_beneficio.show(5)
planes_beneficio.printSchema()

70752
+-----------------+------------------+-------------------------+---------------------------+-----------------+-----------------+----------+-------------+-----------+-------------+--------------+
|IdTipoBeneficio_T|IdAreaDeServicio_T|IdCondicionDePagoCopago_T|IdCondicionDePagoCoseguro_T|IdNivelServicio_T|         IdPlan_T|     Fecha|IdProveedor_T|valorCopago|valorCoseguro|cantidadLimite|
+-----------------+------------------+-------------------------+---------------------------+-----------------+-----------------+----------+-------------+-----------+-------------+--------------+
|              640|          10382017|                       34|                         27|                3|16842FL0070128-03|2017-12-31|        16842|          0|           50|          35.0|
|              335|           5652019|                      238|                         45|                2|14002TN0400104-06|2019-12-31|        14002|          0|            0|           1.0|
|              885|

Ajustar el formato de fechas

In [424]:
planes_beneficio = planes_beneficio.withColumn("Fecha", when(col("Fecha").isNull(), lit("2017-01-01")).otherwise(col("Fecha")))
planes_beneficio = planes_beneficio.withColumn("Fecha", when(length(col("Fecha")) == 7, concat(col("Fecha"), lit("-01"))).otherwise(col("Fecha")))
planes_beneficio = planes_beneficio.withColumn("Fecha", when(length(col("Fecha")) == 4, concat(col("Fecha"), lit("-01-01"))).otherwise(col("Fecha")))
planes_beneficio = planes_beneficio.withColumn("Fecha", to_date(col("Fecha"), "yyyy-MM-dd"))

In [425]:
planes_beneficio = planes_beneficio.withColumn("cantidadLimite", when((col("cantidadLimite").isNull()) | (col("cantidadLimite") == 0), lit(333)).otherwise(col("cantidadLimite")))

planes_beneficio = planes_beneficio.dropDuplicates()

planes_beneficio = planes_beneficio.withColumn('IdPlan_DWH', f.monotonically_increasing_id() + 1)
planes_beneficio = planes_beneficio.withColumn('IdProveedor_DWH', f.monotonically_increasing_id() + 1)

#### Transformación Tipos de Beneficio

In [426]:
print(tipos_beneficio.count())
tipos_beneficio.show(5)
tipos_beneficio.printSchema()

449
+-----------------+--------------------+-------------------+-----+---------------------+-----------------------+----------------------------------------+---------------------------------------+-----+
|IdTipoBeneficio_T|              Nombre|    UnidadDelLimite|EsEHB|EstaCubiertaPorSeguro|TieneLimiteCuantitativo|ExcluidoDelDesembolsoMaximoDentroDeLaRed|ExcluidoDelDesembolsoMaximoFueraDeLaRed|Fecha|
+-----------------+--------------------+-------------------+-----+---------------------+-----------------------+----------------------------------------+---------------------------------------+-----+
|                5|Abortion For Whic...|                   |   No|                   No|                     No|                                      No|                                     No| 2017|
|                5|Abortion For Whic...|                   |   No|                  Yes|                     No|                                      No|                                    Yes| 20

In [427]:
columns = ["EsEHB", "EstaCubiertaPorSeguro", "TieneLimiteCuantitativo", "ExcluidoDelDesembolsoMaximoDentroDeLaRed", "ExcluidoDelDesembolsoMaximoFueraDeLaRed"]

for column in columns:
    count_inconsistent = tipos_beneficio.filter(~col(column).isin("Yes", "No")).count()
    print(f"Número de valores inconsistentes en '{column}': {count_inconsistent}")


Número de valores inconsistentes en 'EsEHB': 22
Número de valores inconsistentes en 'EstaCubiertaPorSeguro': 2
Número de valores inconsistentes en 'TieneLimiteCuantitativo': 13
Número de valores inconsistentes en 'ExcluidoDelDesembolsoMaximoDentroDeLaRed': 0
Número de valores inconsistentes en 'ExcluidoDelDesembolsoMaximoFueraDeLaRed': 0


In [428]:
tipos_beneficio = tipos_beneficio.withColumn("EsEHB", when(col("EsEHB").isin("Yes", "No"), col("EsEHB")).otherwise(lit("No"))) \
                                       .withColumn("EstaCubiertaPorSeguro", when(col("EstaCubiertaPorSeguro").isin("Yes", "No"), col("EstaCubiertaPorSeguro")).otherwise(lit("No"))) \
                                       .withColumn("TieneLimiteCuantitativo", when(col("TieneLimiteCuantitativo").isin("Yes", "No"), col("TieneLimiteCuantitativo")).otherwise(lit("No"))) \
                                       .withColumn("ExcluidoDelDesembolsoMaximoDentroDeLaRed", when(col("ExcluidoDelDesembolsoMaximoDentroDeLaRed").isin("Yes", "No"), col("ExcluidoDelDesembolsoMaximoDentroDeLaRed")).otherwise(lit("No"))) \
                                       .withColumn("ExcluidoDelDesembolsoMaximoFueraDeLaRed", when(col("ExcluidoDelDesembolsoMaximoFueraDeLaRed").isin("Yes", "No"), col("ExcluidoDelDesembolsoMaximoFueraDeLaRed")).otherwise(lit("No")))

tipos_beneficio = tipos_beneficio.dropDuplicates()

tipos_beneficio = tipos_beneficio.withColumn('IdTipoBeneficio_DWH', f.monotonically_increasing_id() + 1)
tipos_beneficio = tipos_beneficio.withColumn('IdCondicionesTipoBeneficio_DWH', f.monotonically_increasing_id() + 1)



In [429]:
tipos_beneficio = tipos_beneficio.withColumn('FechaInicio', concat(col("Fecha"), lit('-01-01')))
windowSpec = Window.partitionBy("IdTipoBeneficio_T").orderBy("FechaInicio")


In [430]:
tipos_beneficio = tipos_beneficio.withColumn("FechaFin",
    F.when(
        F.lead(F.col("IdTipoBeneficio_T")).over(windowSpec) != F.col("IdTipoBeneficio_T"),
        F.lit("2199-12-31")  # Cuando el siguiente IdTipoBeneficio_T es diferente
    ).when(
        (F.lead(F.col("IdTipoBeneficio_T")).over(windowSpec) == F.col("IdTipoBeneficio_T")) &
        (F.lead(F.col("FechaInicio"), 1).over(windowSpec) != F.col("FechaInicio")),
        F.lit(F.date_format(F.date_sub(F.lead("FechaInicio", 1).over(windowSpec), 1), "yyyy-MM-dd"))  # Resta 1 día al inicio del siguiente registro
    ).otherwise(
        F.lit("2199-12-31")  # Por defecto para otros casos
    )
)

tipos_beneficio.show(5)

+-----------------+--------------------+-------------------+-----+---------------------+-----------------------+----------------------------------------+---------------------------------------+-----+-------------------+------------------------------+-----------+----------+
|IdTipoBeneficio_T|              Nombre|    UnidadDelLimite|EsEHB|EstaCubiertaPorSeguro|TieneLimiteCuantitativo|ExcluidoDelDesembolsoMaximoDentroDeLaRed|ExcluidoDelDesembolsoMaximoFueraDeLaRed|Fecha|IdTipoBeneficio_DWH|IdCondicionesTipoBeneficio_DWH|FechaInicio|  FechaFin|
+-----------------+--------------------+-------------------+-----+---------------------+-----------------------+----------------------------------------+---------------------------------------+-----+-------------------+------------------------------+-----------+----------+
|                5|Abortion For Whic...|                   |   No|                   No|                     No|                                      No|                         

#### Transformación condiciones de pago

In [431]:
print(condiciones_pago.count())
condiciones_pago.show()
condiciones_pago.printSchema()

26
+---------------------+--------------------+-----------+
|IdCondicionesDePago_T|         Descripcion|       Tipo|
+---------------------+--------------------+-----------+
|                  187|Copay with deduct...|     Copago|
|                  204|       Copay per Day|     Copago|
|                   45|         Coinsurance|   Coseguro|
|                   85|Copay per Day bef...|     Copago|
|                   18|No Charge after d...|   Coseguro|
|                   27|Coinsurance after...|   Coseguro|
|                  238|               Copay|     Copago|
|                  207|No Charge after d...|     Copago|
|                   68|Copay per Stay af...|     Copago|
|                  153|Copay after deduc...|     Copago|
|                   36|      Not Applicable|   Coseguro|
|                  221|      Copay per Stay|     Copago|
|                  119|Copay per Day wit...|     Copago|
|                  170|Copay before dedu...|     Copago|
|                   51|     

In [432]:
(condiciones_pago.groupBy("Tipo").count()).show()

+-----------+-----+
|       Tipo|count|
+-----------+-----+
|   Copagado|    5|
|Coseguridad|    1|
|     Copago|   15|
|   Coseguro|    5|
+-----------+-----+



In [433]:
condiciones_pago = condiciones_pago.withColumn(
    "Tipo",
    when(lower(col("Tipo")).startswith("cop"), lit("Copago"))
    .when(lower(col("Tipo")).startswith("cos"), lit("Coseguro"))
    .otherwise(col("Tipo"))
)
condiciones_pago = condiciones_pago.dropDuplicates()
condiciones_pago = condiciones_pago.withColumn('IdCondicionesDePago_DWH', f.monotonically_increasing_id() + 1)


#### Creación de tablas

#### Dimensiones

In [434]:
# Guardar los datos en las tablas creadas
#guardar_db(dest_db_connection_string, areas_servicio, "AreasDeServicio", db_user, db_psswd)
#guardar_db(dest_db_connection_string, condiciones_pago, "CondicionesDePago", db_user, db_psswd)
#guardar_db(dest_db_connection_string, tipos_beneficio, "TiposBeneficio", db_user, db_psswd)
#guardar_db(dest_db_connection_string, niveles_servicio, "NivelesDeServicio", db_user, db_psswd)


In [435]:
dim_areas_servicio = areas_servicio.select(
    col("IdAreaDeServicio_DWH"),
    col("IdAreaDeServicio_T"),
    col("NombreAreaDeServicio").alias("Nombre"),
    col("Fecha").alias("AnnoCreacion")
)
desconocido_area = spark.createDataFrame([(0, 0,"Desconocido", "1900-01-01")], ["IdAreaDeServicio_DWH", "IdAreaDeServicio_T","Nombre", "AnnoCreacion"])
dim_areas_servicio = dim_areas_servicio.union(desconocido_area)

print(dim_areas_servicio.count())
dim_areas_servicio.show(5)
dim_areas_servicio.printSchema()

268445
+--------------------+------------------+--------------------+------------+
|IdAreaDeServicio_DWH|IdAreaDeServicio_T|              Nombre|AnnoCreacion|
+--------------------+------------------+--------------------+------------+
|                   1|         106862021|Medica Insure9307...|        2021|
|                   2|         112302018|Statewide Medical...|        2018|
|                   3|          75752018|Avera Health Plan...|        2018|
|                   4|          99152017|Ambetter of Magno...|        2017|
|                   5|         115692019|Home State Health...|        2019|
+--------------------+------------------+--------------------+------------+
only showing top 5 rows

root
 |-- IdAreaDeServicio_DWH: long (nullable = true)
 |-- IdAreaDeServicio_T: long (nullable = true)
 |-- Nombre: string (nullable = true)
 |-- AnnoCreacion: string (nullable = true)



In [436]:
dim_condiciones_pago = condiciones_pago.select(
    col("IdCondicionesDePago_DWH"),
    col("IdCondicionesDePago_T"),
    col("Descripcion"),
    col("Tipo")
).distinct()

print(dim_condiciones_pago.count())
dim_condiciones_pago.show(5)
dim_condiciones_pago.printSchema()

20
+-----------------------+---------------------+--------------------+--------+
|IdCondicionesDePago_DWH|IdCondicionesDePago_T|         Descripcion|    Tipo|
+-----------------------+---------------------+--------------------+--------+
|                      1|                  221|      Copay per Stay|  Copago|
|                      2|                   51|           No Charge|  Copago|
|                      3|                   36|      Not Applicable|Coseguro|
|                      4|                   27|Coinsurance after...|Coseguro|
|                      5|                   17|Copay per Day aft...|  Copago|
+-----------------------+---------------------+--------------------+--------+
only showing top 5 rows

root
 |-- IdCondicionesDePago_DWH: long (nullable = false)
 |-- IdCondicionesDePago_T: integer (nullable = true)
 |-- Descripcion: string (nullable = true)
 |-- Tipo: string (nullable = true)



In [437]:
dim_proveedor = planes_beneficio.select(
    col("IdProveedor_DWH"),
    col("IdProveedor_T")
).distinct()

print(dim_proveedor.count())
dim_proveedor.show()
dim_proveedor.printSchema()

70752
+---------------+-------------+
|IdProveedor_DWH|IdProveedor_T|
+---------------+-------------+
|            247|        38345|
|            429|        91450|
|           1432|        36194|
|           1577|        30252|
|           1715|        36096|
|           1933|        12379|
|           1958|        14002|
|           2091|        67243|
|           2135|        27833|
|           2162|        87247|
|           2369|        94529|
|           2530|        96751|
|           2591|        67243|
|           2668|        33602|
|           2833|        60612|
|           2949|        28162|
|           3261|        37833|
|           3344|        12379|
|           3551|        96751|
|           3667|        83761|
+---------------+-------------+
only showing top 20 rows

root
 |-- IdProveedor_DWH: long (nullable = false)
 |-- IdProveedor_T: integer (nullable = true)



In [438]:
dim_tipo_beneficio = tipos_beneficio.select(
    col("IdTipoBeneficio_DWH"),
    col("IdTipoBeneficio_T"),
    col("Nombre")
)
print(dim_tipo_beneficio.count())
dim_tipo_beneficio.show()
dim_tipo_beneficio.printSchema()

345
+-------------------+-----------------+--------------------+
|IdTipoBeneficio_DWH|IdTipoBeneficio_T|              Nombre|
+-------------------+-----------------+--------------------+
|                  1|              480|Mental/behavioral...|
|                  2|              235|Durable Medical E...|
|                  3|               10|   Accidental Dental|
|                  4|              425|Inpatient Rehabil...|
|                  5|              780|Rehabilitative Oc...|
|                  6|              980|          Transplant|
|                  7|              890|Substance Abuse D...|
|                  8|              140|     Clinical Trials|
|                  9|              335|        Hearing Aids|
|                 10|              585|        Oral Surgery|
|                 11|              610|Osteoporosis Trea...|
|                 12|              730|  Preventative Drugs|
|                 13|              225|            Dialysis|
|                 14

In [439]:
dim_niveles_servicio = niveles_servicio.select(
    col("IdNivelDeServicio_DWH"),
    col("IdNivelDeServicio_T"),
    col("Descripcion")
)
print(dim_niveles_servicio.count())
dim_niveles_servicio.show()
dim_niveles_servicio.printSchema()

3
+---------------------+-------------------+---------------+
|IdNivelDeServicio_DWH|IdNivelDeServicio_T|    Descripcion|
+---------------------+-------------------+---------------+
|                    1|                  1|        Nivel 1|
|                    2|                  2|        Nivel 2|
|                    3|                  3|Fuera de la red|
+---------------------+-------------------+---------------+

root
 |-- IdNivelDeServicio_DWH: integer (nullable = true)
 |-- IdNivelDeServicio_T: integer (nullable = true)
 |-- Descripcion: string (nullable = true)



In [440]:
dim_geografia = areas_servicio.select(
    col("IdGeografia_DWH"),
    col("IdGeografia_T"),
    col("Estado"),
    col("Condado"),
    col("Area").alias("AreaAct"),    
    col("Densidad").alias("DensidadAct"),
    col("PoblacionAct")
).distinct()
print(dim_geografia.count())
dim_geografia.show()
dim_geografia.printSchema()


268444
+---------------+-------------+------------+-----------------+-------+-----------+------------+
|IdGeografia_DWH|IdGeografia_T|      Estado|          Condado|AreaAct|DensidadAct|PoblacionAct|
+---------------+-------------+------------+-----------------+-------+-----------+------------+
|            148|        39175|        Ohio|   Wyandot County|  405.0|       54.0|       21708|
|            295|        40097|    Oklahoma|     Mayes County|  656.0|       60.0|       39159|
|            972|        48085|       Texas|    Collin County|  848.0|     1308.0|     1109462|
|           1125|        26161|    Michigan| Washtenaw County|  723.0|      511.0|      369390|
|           1194|        28089| Mississippi|   Madison County|  719.0|      153.0|      109813|
|           1608|        48319|       Texas|     Mason County|  932.0|        4.0|        3943|
|           1768|        17013|    Illinois|   Calhoun County|  253.0|       17.0|        4369|
|           1958|        48141|  

In [441]:
fecha = planes_beneficio.select(col("Fecha").alias("IdFechaEmision")).distinct()
fecha = fecha.withColumn("Anno", year(col("IdFechaEmision"))) \
                   .withColumn("Mes", month(col("IdFechaEmision"))) \
                   .withColumn("Dia", dayofmonth(col("IdFechaEmision")))

print(fecha.count())
fecha.show()
fecha.printSchema()


6
+--------------+----+----+----+
|IdFechaEmision|Anno| Mes| Dia|
+--------------+----+----+----+
|    2017-12-31|2017|  12|  31|
|          null|null|null|null|
|    2018-12-31|2018|  12|  31|
|    2020-12-31|2020|  12|  31|
|    2021-12-31|2021|  12|  31|
|    2019-12-31|2019|  12|  31|
+--------------+----+----+----+

root
 |-- IdFechaEmision: date (nullable = true)
 |-- Anno: integer (nullable = true)
 |-- Mes: integer (nullable = true)
 |-- Dia: integer (nullable = true)



In [442]:
dim_mini_condiciones = tipos_beneficio.select(
    col("IdCondicionesTipoBeneficio_DWH"),
    col("EsEHB"),
    col("EstaCubiertaPorSeguro"),
    col("TieneLimiteCuantitativo"),
    col("ExcluidoDelDesembolsoMaximoDentroDeLaRed"),
    col("ExcluidoDelDesembolsoMaximoFueraDeLaRed")
).distinct()

print(dim_mini_condiciones.count())
dim_mini_condiciones.show()
dim_mini_condiciones.printSchema()


345
+------------------------------+-----+---------------------+-----------------------+----------------------------------------+---------------------------------------+
|IdCondicionesTipoBeneficio_DWH|EsEHB|EstaCubiertaPorSeguro|TieneLimiteCuantitativo|ExcluidoDelDesembolsoMaximoDentroDeLaRed|ExcluidoDelDesembolsoMaximoFueraDeLaRed|
+------------------------------+-----+---------------------+-----------------------+----------------------------------------+---------------------------------------+
|                            56|  Yes|                  Yes|                     No|                                      No|                                     No|
|                           105|   No|                  Yes|                     No|                                      No|                                     No|
|                           263|  Yes|                  Yes|                     No|                                      No|                                    Yes|


In [443]:
asociacion_area_servicio = areas_servicio.select(
    col("IdAreaDeServicio_DWH"),
    col("IdGeografia_DWH")
).distinct()
print(asociacion_area_servicio.count())
asociacion_area_servicio.show()
asociacion_area_servicio.printSchema()


268444
+--------------------+---------------+
|IdAreaDeServicio_DWH|IdGeografia_DWH|
+--------------------+---------------+
|                 346|            346|
|                 435|            435|
|                 753|            753|
|                 787|            787|
|                1167|           1167|
|                1395|           1395|
|                1508|           1508|
|                1909|           1909|
|                2005|           2005|
|                2226|           2226|
|                2382|           2382|
|                2941|           2941|
|                3416|           3416|
|                4294|           4294|
|                4640|           4640|
|                4925|           4925|
|                5103|           5103|
|                5559|           5559|
|                5636|           5636|
|                5983|           5983|
+--------------------+---------------+
only showing top 20 rows

root
 |-- IdAreaDeServicio_DWH:

In [444]:
hecho_hist_condiciones = tipos_beneficio.select(
    col("IdTipoBeneficio_DWH"),
    col("IdCondicionesTipoBeneficio_DWH"),
    lit(1).alias("Cambio"),
    col("FechaInicio"),
    col("FechaFin")
)
print(hecho_hist_condiciones.count())
hecho_hist_condiciones.show()
hecho_hist_condiciones.printSchema()


345
+-------------------+------------------------------+------+-----------+----------+
|IdTipoBeneficio_DWH|IdCondicionesTipoBeneficio_DWH|Cambio|FechaInicio|  FechaFin|
+-------------------+------------------------------+------+-----------+----------+
|                 46|                            46|     1| 2017-01-01|2017-12-31|
|                207|                           207|     1| 2018-01-01|2019-12-31|
|                 36|                            36|     1| 2020-01-01|2199-12-31|
|                  3|                             3|     1| 2017-01-01|2017-12-31|
|                151|                           151|     1| 2018-01-01|2018-12-31|
|                299|                           299|     1| 2019-01-01|2020-12-31|
|                 64|                            64|     1| 2021-01-01|2199-12-31|
|                331|                           331|     1| 2017-01-01|2017-12-31|
|                106|                           106|     1| 2018-01-01|2018-12-31|


### Hechos

In [508]:
# Mostrar las columnas de los dataframes involucrados
print("Columns in proveedor:", dim_proveedor.columns)
print("Columns in tipobeneficio:", dim_tipo_beneficio.columns)
print("Columns in nivelesbeneficio:", dim_niveles_servicio.columns)
print("Columns in condiciones_pago:", dim_condiciones_pago.columns)
print("Columns in geografia:",dim_geografia.columns)
print("Columns in fecha:",fecha.columns)
print("Columns in mini condiciones:",dim_mini_condiciones.columns)
print("Columns in areaserviio:",asociacion_area_servicio.columns)
print("Columns in hechohist:",hecho_hist_condiciones.columns)

Columns in proveedor: ['IdProveedor_DWH', 'IdProveedor_T']
Columns in tipobeneficio: ['IdTipoBeneficio_DWH', 'IdTipoBeneficio_T', 'Nombre']
Columns in nivelesbeneficio: ['IdNivelDeServicio_DWH', 'IdNivelDeServicio_T', 'Descripcion']
Columns in condiciones_pago: ['IdCondicionesDePago_DWH', 'IdCondicionesDePago_T', 'Descripcion', 'Tipo']
Columns in geografia: ['IdGeografia_DWH', 'IdGeografia_T', 'Estado', 'Condado', 'AreaAct', 'DensidadAct', 'PoblacionAct']
Columns in fecha: ['IdFechaEmision', 'Anno', 'Mes', 'Dia']
Columns in mini condiciones: ['IdCondicionesTipoBeneficio_DWH', 'EsEHB', 'EstaCubiertaPorSeguro', 'TieneLimiteCuantitativo', 'ExcluidoDelDesembolsoMaximoDentroDeLaRed', 'ExcluidoDelDesembolsoMaximoFueraDeLaRed']
Columns in areaserviio: ['IdAreaDeServicio_DWH', 'IdGeografia_DWH']
Columns in hechohist: ['IdTipoBeneficio_DWH', 'IdCondicionesTipoBeneficio_DWH', 'Cambio', 'FechaInicio', 'FechaFin']


In [525]:
dim_condicion_pago = dim_condicion_pago.withColumnRenamed("Descripcion", "DescripcionCondicionPago")

# Alias para los dataframes
prov_alias = dim_proveedor.alias("p")
tb_alias = dim_tipo_beneficio.alias("tb")
ns_alias = dim_niveles_servicio.alias("ns")
cp_coseguro_alias = dim_condiciones_pago.filter(dim_condicion_pago["Tipo"] == "Coseguro").alias("cp_coseguro")
cp_copago_alias = dim_condiciones_pago.filter(dim_condicion_pago["Tipo"] == "Copago").alias("cp_copago")
geo_alias = dim_geografia.alias("geo")
fecha_alias = fecha.alias("fecha")
mini_cond_alias = dim_mini_condiciones.alias("mc")
area_serv_alias = asociacion_area_servicio.alias("as")
hecho_hist_alias = hecho_hist_condiciones.alias("hh")



In [526]:
fact_table = tb_alias.join(
    prov_alias,
    tb_alias["IdTipoBeneficio_DWH"] == prov_alias["IdProveedor_DWH"],
    "left"
).select(
    tb_alias["IdTipoBeneficio_DWH"].alias("IdTipoBeneficio_DWH"),
    tb_alias["IdTipoBeneficio_T"].alias("IdTipoBeneficio_T"),
    tb_alias["Nombre"].alias("NombreTipoBeneficio"),
    prov_alias["IdProveedor_T"].alias("IdProveedor")
)

# Mostrar el esquema y datos después del primer join
fact_table.printSchema()
fact_table.show()

root
 |-- IdTipoBeneficio_DWH: long (nullable = false)
 |-- IdTipoBeneficio_T: integer (nullable = true)
 |-- NombreTipoBeneficio: string (nullable = true)
 |-- IdProveedor: integer (nullable = true)

+-------------------+-----------------+--------------------+-----------+
|IdTipoBeneficio_DWH|IdTipoBeneficio_T| NombreTipoBeneficio|IdProveedor|
+-------------------+-----------------+--------------------+-----------+
|                 19|              580|Off-label Prescri...|      33602|
|                  7|              890|Substance Abuse D...|      96751|
|                  6|              980|          Transplant|      30751|
|                  9|              335|        Hearing Aids|      33602|
|                 17|              110|Breast Implant Re...|      36194|
|                  5|              780|Rehabilitative Oc...|      37833|
|                  1|              480|Mental/behavioral...|      33602|
|                 10|              585|        Oral Surgery|      216

In [527]:
fact_table = fact_table.join(
    ns_alias,
    fact_table["IdTipoBeneficio_DWH"] == ns_alias["IdNivelDeServicio_DWH"],
    "left"
).select(
    fact_table["IdTipoBeneficio_DWH"],
    fact_table["IdTipoBeneficio_T"],
    fact_table["NombreTipoBeneficio"],
    fact_table["IdProveedor"],
    ns_alias["IdNivelDeServicio_T"].alias("IdNivelDeServicio"),
    ns_alias["Descripcion"].alias("DescripcionNivelServicio")
)
# Mostrar el esquema y datos después del segundo join
fact_table.printSchema()
fact_table.show()

root
 |-- IdTipoBeneficio_DWH: long (nullable = false)
 |-- IdTipoBeneficio_T: integer (nullable = true)
 |-- NombreTipoBeneficio: string (nullable = true)
 |-- IdProveedor: integer (nullable = true)
 |-- IdNivelDeServicio: integer (nullable = true)
 |-- DescripcionNivelServicio: string (nullable = true)

+-------------------+-----------------+--------------------+-----------+-----------------+------------------------+
|IdTipoBeneficio_DWH|IdTipoBeneficio_T| NombreTipoBeneficio|IdProveedor|IdNivelDeServicio|DescripcionNivelServicio|
+-------------------+-----------------+--------------------+-----------+-----------------+------------------------+
|                 19|              580|Off-label Prescri...|      33602|             null|                    null|
|                  7|              890|Substance Abuse D...|      96751|             null|                    null|
|                  6|              980|          Transplant|      30751|             null|                    nul

In [529]:
fact_table = fact_table.join(
    cp_coseguro_alias,
    fact_table["IdTipoBeneficio_DWH"] == cp_coseguro_alias["IdCondicionesDePago_DWH"],
    "left"
).select(
    fact_table["IdTipoBeneficio_DWH"],
    fact_table["IdTipoBeneficio_T"],
    fact_table["NombreTipoBeneficio"],
    fact_table["IdProveedor"],
    fact_table["IdNivelDeServicio"],
    fact_table["DescripcionNivelServicio"],
    cp_coseguro_alias["IdCondicionesDePago_T"].alias("IdCondicionDePagoCoseguro"),
    cp_coseguro_alias["DescripcionCondicionPago"].alias("DescripcionCondicionPagoCoseguro"),
    cp_coseguro_alias["Tipo"].alias("TipoCondicionPagoCoseguro")
)

AnalysisException: Cannot resolve column name "DescripcionCondicionPago" among (IdCondicionesDePago_DWH, IdCondicionesDePago_T, Descripcion, Tipo)

In [513]:
fact_table = fact_table.withColumnRenamed("DescripcionCondicionPagoCoseguro", "DescripcionCoseguro").withColumnRenamed("IdCondicionDePagoCoseguro", "IdCoseguro")


In [515]:
fact_table = fact_table.join(
    cp_copago_alias,
    fact_table["IdTipoBeneficio_DWH"] == cp_copago_alias["IdCondicionesDePago_DWH"],
    "left"
).select(
    fact_table["IdTipoBeneficio_DWH"],
    fact_table["IdTipoBeneficio_T"],
    fact_table["NombreTipoBeneficio"],
    fact_table["IdProveedor"],
    fact_table["IdNivelDeServicio"],
    fact_table["DescripcionNivelServicio"],
    fact_table["IdCoseguro"],
    fact_table["DescripcionCoseguro"],
    fact_table["TipoCondicionPagoCoseguro"],
    cp_copago_alias["IdCondicionesDePago_T"].alias("IdCondicionDePagoCopago"),
    cp_copago_alias["DescripcionCondicionPago"].alias("DescripcionCondicionPagoCopago"),
    cp_copago_alias["Tipo"].alias("TipoCondicionPagoCopago")
)
# Mostrar el esquema y datos después del segundo join
fact_table.printSchema()
fact_table.show()

AnalysisException: Cannot resolve column name "DescripcionCondicionPago" among (IdCondicionesDePago_DWH, IdCondicionesDePago_T, Descripcion, Tipo)

In [ ]:
fact_table = fact_table.join(
    geo_alias,
    fact_table["IdTipoBeneficio_T"] == geo_alias["IdGeografia_DWH"],
    "inner"
).select(
    fact_table["IdProveedor_T"],
    fact_table["IdTipoBeneficio_T"],
    fact_table["Nombre"],
    fact_table["IdNivelDeServicio_T"],
    fact_table["Descripcion"],
    fact_table["IdCondicionesDePago_T"],
    fact_table["DescripcionCondicionPago"],
    fact_table["Tipo"],
    geo_alias["IdGeografia_T"],
    geo_alias["Estado"],
    geo_alias["Condado"],
    geo_alias["AreaAct"],
    geo_alias["DensidadAct"],
    geo_alias["PoblacionAct"]
)

# Mostrar el esquema y datos después del cuarto join
fact_table.printSchema()
fact_table.show()


In [408]:
fact_table = fact_table.join(
    dim_niveles_servicio.alias("ns"),
    fact_table["IdTipoBeneficio_T"] == dim_niveles_servicio["IdNivelDeServicio_T"],
    "inner"
).select(
    "fact_table.*",
    "ns.IdNivelDeServicio_T",
    "ns.Descripcion_T"
)

AnalysisException: cannot resolve 'fact_table.*' given input columns 'IdProveedor_T, IdTipoBeneficio_T, Nombre, IdNivelDeServicio_DWH, IdNivelDeServicio_T, Descripcion'

In [345]:
# Mostrar los nombres de las columnas del dataframe planes_beneficio
print(planes_beneficio.columns)



['IdTipoBeneficio_T_Plan', 'IdAreaDeServicio_T_Plan', 'IdCondicionDePagoCopago_T_Plan', 'IdCondicionDePagoCoseguro_T_Plan', 'IdNivelDeServicio_T_Plan', 'IdPlan_DWH', 'Fecha', 'IdProveedor_DWH', 'valorCopago', 'valorCoseguro', 'cantidadLimite', 'IdPlan_DWH', 'IdProveedor_DWH']


In [534]:

# Join con la dimensión de Tipos de Beneficio y Proveedor
fact_table = tb_alias.join(
    prov_alias,
    tb_alias["IdTipoBeneficio_DWH"] == prov_alias["IdProveedor_DWH"],
    "left"
).select(
    tb_alias["IdTipoBeneficio_DWH"].alias("IdTipoBeneficio_DWH"),
    tb_alias["IdTipoBeneficio_T"].alias("IdTipoBeneficio_T"),
    tb_alias["Nombre"].alias("NombreTipoBeneficio"),
    prov_alias["IdProveedor_T"].alias("IdProveedor")
)

# Mostrar el esquema y datos después del primer join
fact_table.printSchema()
fact_table.show()

# Join con la dimensión de Niveles de Servicio
fact_table = fact_table.join(
    ns_alias,
    fact_table["IdTipoBeneficio_DWH"] == ns_alias["IdNivelDeServicio_DWH"],
    "left"
).select(
    fact_table["IdTipoBeneficio_DWH"],
    fact_table["IdTipoBeneficio_T"],
    fact_table["NombreTipoBeneficio"],
    fact_table["IdProveedor"],
    ns_alias["IdNivelDeServicio_T"].alias("IdNivelDeServicio"),
    ns_alias["Descripcion"].alias("DescripcionNivelServicio")
)

# Mostrar el esquema y datos después del segundo join
fact_table.printSchema()
fact_table.show()

# Join con la dimensión de Condiciones de Pago (Coseguro)
fact_table = fact_table.join(
    cp_coseguro_alias,
    fact_table["IdTipoBeneficio_DWH"] == cp_coseguro_alias["IdCondicionesDePago_DWH"],
    "left"
).select(
    fact_table["IdTipoBeneficio_DWH"],
    fact_table["IdTipoBeneficio_T"],
    fact_table["NombreTipoBeneficio"],
    fact_table["IdProveedor"],
    fact_table["IdNivelDeServicio"],
    fact_table["DescripcionNivelServicio"],
    cp_coseguro_alias["IdCondicionesDePago_T"].alias("IdCondicionDePagoCoseguro"),
    cp_coseguro_alias["DescripcionCondicionPago"].alias("DescripcionCondicionPagoCoseguro"),
    cp_coseguro_alias["Tipo"].alias("TipoCondicionPagoCoseguro")
)

# Renombrar las columnas después del join para evitar ambigüedades
fact_table = fact_table.withColumnRenamed("DescripcionCondicionPagoCoseguro", "DescripcionCoseguro").withColumnRenamed("IdCondicionDePagoCoseguro", "IdCoseguro")

# Mostrar el esquema y datos después del tercer join
fact_table.printSchema()
fact_table.show()

# Join con la dimensión de Condiciones de Pago (Copago)
fact_table = fact_table.join(
    cp_copago_alias,
    fact_table["IdTipoBeneficio_DWH"] == cp_copago_alias["IdCondicionesDePago_DWH"],
    "left"
).select(
    fact_table["IdTipoBeneficio_DWH"],
    fact_table["IdTipoBeneficio_T"],
    fact_table["NombreTipoBeneficio"],
    fact_table["IdProveedor"],
    fact_table["IdNivelDeServicio"],
    fact_table["DescripcionNivelServicio"],
    fact_table["IdCoseguro"],
    fact_table["DescripcionCoseguro"],
    fact_table["TipoCondicionPagoCoseguro"],
    cp_copago_alias["IdCondicionesDePago_T"].alias("IdCondicionDePagoCopago"),
    cp_copago_alias["DescripcionCondicionPago"].alias("DescripcionCondicionPagoCopago"),
    cp_copago_alias["Tipo"].alias("TipoCondicionPagoCopago")
)

# Renombrar las columnas después del join para evitar ambigüedades
fact_table = fact_table.withColumnRenamed("DescripcionCondicionPagoCopago", "DescripcionCopago")

# Mostrar el esquema y datos después del cuarto join
fact_table.printSchema()
fact_table.show()

# Join con la dimensión de Geografía
fact_table = fact_table.join(
    geo_alias,
    fact_table["IdTipoBeneficio_DWH"] == geo_alias["IdGeografia_DWH"],
    "left"
).select(
    fact_table["IdTipoBeneficio_DWH"],
    fact_table["IdTipoBeneficio_T"],
    fact_table["NombreTipoBeneficio"],
    fact_table["IdProveedor"],
    fact_table["IdNivelDeServicio"],
    fact_table["DescripcionNivelServicio"],
    fact_table["IdCoseguro"],
    fact_table["DescripcionCoseguro"],
    fact_table["TipoCondicionPagoCoseguro"],
    fact_table["IdCondicionDePagoCopago"],
    fact_table["DescripcionCopago"],
    fact_table["TipoCondicionPagoCopago"],
    geo_alias["IdGeografia_T"].alias("IdGeografia"),
    geo_alias["Estado"],
    geo_alias["Condado"],
    geo_alias["AreaAct"],
    geo_alias["DensidadAct"],
    geo_alias["PoblacionAct"]
)

# Mostrar el esquema y datos después del quinto join
fact_table.printSchema()
fact_table.show()

# Join con la dimensión de Fecha
fact_table = fact_table.join(
    fecha_alias,
    fact_table["IdTipoBeneficio_DWH"] == fecha_alias["IdFechaEmision"],
    "left"
).select(
    fact_table["IdTipoBeneficio_DWH"],
    fact_table["IdTipoBeneficio_T"],
    fact_table["NombreTipoBeneficio"],
    fact_table["IdProveedor"],
    fact_table["IdNivelDeServicio"],
    fact_table["DescripcionNivelServicio"],
    fact_table["IdCoseguro"],
    fact_table["DescripcionCoseguro"],
    fact_table["TipoCondicionPagoCoseguro"],
    fact_table["IdCondicionDePagoCopago"],
    fact_table["DescripcionCopago"],
    fact_table["TipoCondicionPagoCopago"],
    fact_table["IdGeografia"],
    fact_table["Estado"],
    fact_table["Condado"],
    fact_table["AreaAct"],
    fact_table["DensidadAct"],
    fact_table["PoblacionAct"],
    fecha_alias["Anno"],
    fecha_alias["Mes"],
    fecha_alias["Dia"]
)

# Mostrar el esquema y datos después del sexto join
fact_table.printSchema()
fact_table.show()


root
 |-- IdTipoBeneficio_DWH: long (nullable = false)
 |-- IdTipoBeneficio_T: integer (nullable = true)
 |-- NombreTipoBeneficio: string (nullable = true)
 |-- IdProveedor: integer (nullable = true)

+-------------------+-----------------+--------------------+-----------+
|IdTipoBeneficio_DWH|IdTipoBeneficio_T| NombreTipoBeneficio|IdProveedor|
+-------------------+-----------------+--------------------+-----------+
|                 19|              580|Off-label Prescri...|      33602|
|                  7|              890|Substance Abuse D...|      96751|
|                  6|              980|          Transplant|      30751|
|                  9|              335|        Hearing Aids|      33602|
|                 17|              110|Breast Implant Re...|      36194|
|                  5|              780|Rehabilitative Oc...|      37833|
|                  1|              480|Mental/behavioral...|      33602|
|                 10|              585|        Oral Surgery|      216

AnalysisException: Cannot resolve column name "DescripcionCondicionPago" among (IdCondicionesDePago_DWH, IdCondicionesDePago_T, Descripcion, Tipo)